In [1]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict

from transformers import BertTokenizer,BertModel,BertTokenizerFast,DebertaTokenizer,DebertaTokenizerFast, AutoTokenizer
from utils.MIND import MIND
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [15]:
t = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache=config.path + "bert_cache/")
# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache=config.path + "bert_cache/")

In [2]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.signal_length = 10
# config.scale = 'large'
# config.impr_size = 100
# config.mode = 'test'

config.bert = 'microsoft/deberta-base'
config.embedding = 'deberta'

# config.bert = 'bert-base-uncased'
# config.embedding = 'bert'

manager = Manager(config)
path = manager.path + 'MIND/MINDdemo_train/'
a = MIND(manager, path + 'news.tsv', path + 'behaviors.tsv')

[2021-09-09 21:04:27,051] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/deberta/MINDdemo_train/10/behaviors..pkl
[2021-09-09 21:04:27,073] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/deberta/MINDdemo_train/news.pkl
[2021-09-09 21:04:27,941] INFO (utils.utils) deduplicating...


In [16]:
tokens = t.tokenize("[CLS] I love you embeddings.")
tokens

['[CLS]', 'ĠI', 'Ġlove', 'Ġyou', 'Ġembed', 'd', 'ings', '.']

In [15]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/deberta/MINDdemo_train/news.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

In [13]:
t.convert_tokens_to_ids([i for i in r"[.&*()+=/\<>,!?;:~`@#$%^]"])

[1031,
 1012,
 1004,
 1008,
 1006,
 1007,
 1009,
 1027,
 1013,
 1032,
 1026,
 1028,
 1010,
 999,
 1029,
 1025,
 1024,
 1066,
 1036,
 1030,
 1001,
 1002,
 1003,
 1034,
 1033]